## Técnicas Computacionais

Para a escolha de chaves e a encriptação, basta utilizar um algoritmo de geração de primos pseudo aleatórios e realizar o cálculo de um quadrado e um módulo. Para a decriptação, precisamos fazer uso de algumas técnicas computacionais conhecidas.

### Encontrando as Chaves

Primeiro, encontramos uma chave pelo seguinte processo:

Para a escolha de chaves e a encriptação, basta utilizar um algoritmo de geração de primos pseudo aleatórios e realizar o cálculo de um quadrado e um módulo. Para a decriptação, precisamos fazer uso de algumas técnicas computacionais conhecidas.

1. Escolher 2 primos grandes $p$ e $q$ tais que $p \equiv q \equiv 3 \mod 4$

2. Calcular $n$ tal que $n = pq$

Os números calculados serão as chaves:

* Chave Pública: Um inteiro $n = pq$

* Chave Privada: Par ordenado $(p, q)$.

In [153]:
from Crypto.Util import number

p = q = 0
while(p % 4 != 3):
    p = number.getPrime(32)
while(q % 4 != 3):
    q = number.getPrime(32)

n = p*q

print("Chave Pública: ", n)
print("Chave Privada: ", (p, q))

Chave Pública:  6647767211314390177
Chave Privada:  (2461998859, 2700150403)


### Encriptação

Lembrando que Bob tem acesso apenas à chave pública $n$, para encriptar uma mensagem $M$, ele seguiria dois passos:

1. Transformar $M$ em um número qualquer tal que $m < n$
2. Encontrar $c$ tal que $c \equiv m^2 \mod n$

O $c$ encontrado é a mensagem criptografada.

In [154]:
import base64

def decode(m):
    m       = str(m)
    message = ""
    for i in range(0, len(m)-1, 2):
        c        = m[i]+m[i+1]
        message += chr(int(c))
    
    return message

def encode(message):
    m = ""
    for c in message:
        a = ord(c)
        m += str(a)
    
    return int(m)

In [160]:
print("Insira a mensagem: ")
m = encode(input())

c = (m ** 2) % n
print("\nMensagem transformada em inteiro: ")
print(m)
print("\nEssa é a mensagem cifrada: ")
print(c)

Insira a mensagem: 


 AAAAA



Mensagem transformada em inteiro: 
6565656565

Essa é a mensagem cifrada: 
3221242861641258163


---

### Decriptação

Para decriptar a mensagem, Alice tem acesso à $c$ e $(p,q)$ e irá encontrar o valor original $m$. A decriptação segue três passos:


### Primeira Parte - Exponenciação Rápida

1. Calcular dois valores $m_p$ e $m_q$ tais que:

$$
\begin{align}
m_p &\equiv c^{\frac{p+1}{4}} \mod p \\
m_q &\equiv c^{\frac{q+1}{4}} \mod q
\end{align}
$$

Para a primeira parte da decriptação, basta a realização de uma exponenciação modular, que pode ser resolvida por meio de um algoritmo de exponenciação rápida em tempo logarítmico no tamanho do expoente.

$$
x^{y} = (x^2)^{\frac{y}{2}}
$$

In [142]:
def fexp(x, y, m): # X ^ Y mod m
    if y == 0: return 1
    ans = fexp(x * x % m, int(y/2), m)
    
    if y%2 == 1:
        return x * ans % m
    else:
        return ans

#### Calculando:

In [161]:
m_p = fexp(c, int((p+1)/4), p)
m_q = fexp(c, int((q+1)/4), q)

print("M_p: ", m_p)
print("M_q: ", m_q)

M_p:  820340012
M_q:  1534794644


---

### Segunda Parte - Algoritmo Extendido de Euclides

2. Encontrar valores de $x_p$ e $x_q$ que satisfaçam a seguinte equação diofantina:

$$
p \cdot x_p + q \cdot x_q = 1
$$

Para a segunda parte da decriptação, precisamos resolver uma equação diofantina. Para isso, faremos uso do algoritmo extendido Euclides. O algoritmo extendido de Euclides encontra, dados dois números $a$ e $b$, o mínimo divisor comum entre esses dois números e $x$ e $y$ tais que $p \cdot x + q \cdot y = \gcd(p, q)$. Como $p$ e $q$ são primos, $\gcd(p, q) = 1$ e portanto o Algoritmo Extendido de Euclides retorna a resposta da equação diofantina.

In [157]:
def gcdExtended(a, b): 
    if a == 0 :  
        return b, 0, 1
             
    gcd, x1, y1 = gcdExtended(b%a, a)
    
    x = y1 - (b//a) * x1 
    y = x1 
     
    return gcd, x, y  

#### Calculando:

In [162]:
_, x_p, x_q = gcdExtended(p, q)
print("X_p: ", x_p)
print("X_q: ", x_q)
print("\nX_p * p + X_q * q :\n")
print(x_p, "*", p, "+", x_q, "*", q, "=", x_p*p + x_q*q)

X_p:  -332786680
X_q:  303435107

X_p * p + X_q * q :

-332786680 * 2461998859 + 303435107 * 2700150403 = 1


---

## Terceira Parte - Teorema Chinês do Resto
3. Encontrar quatro possiveis mensagens originais resolvendo as seguintes equações:
   $$
   \begin{align}
   m_1 &\equiv x_p \cdot p \cdot m_q + x_q \cdot q \cdot m_p \mod n \\
   m_2 &\equiv n - m_1 \\
   m_3 &\equiv x_p \cdot p \cdot m_q - x_q \cdot q \cdot m_p \mod n \\
   m_4 &\equiv n - m_3
   \end{align}
   $$

Para a parte final da decriptação, queremos calcular $m_1$, $m_2$, $m_3$ e $m_4$ tendo em mãos $p$, $m_p$, $x_p$, $q$, $m_q$ e $x_q$. Basta realizarmos as operações, porém, como esses números podem ser muito grandes, em aplicações reais é utilizado o Teorema Chinês do Resto para realizar os cálculos de maneira eficiente. O Teorema Chinês do Resto é muito utilizado na computação de cálculos com inteiros muito grandes e diz que, sabendo os restos da divisão euclidiana de um inteiro $n$ por um conjunto de inteiros é possível determinar o resto da divisão de $n$ pelo produto desses inteiros, desde que todos esses inteiros sejam primos entre si. Para essa demonstração, apenas faremos as operações

In [163]:
ans = [0 for i in range(4)]

ans[0] = (x_p * p * m_q + x_q * q * m_p) % n
ans[1] = n - ans[0]
ans[2] = (x_p * p * m_q - x_q * q * m_p) % n
ans[3] = n - ans[2]

print("Números encontrados: \n")
for i in range(4):
    print("M", i+1, ": ", ans[i], sep="")
    
print("\nMensagens encontradas: \n")
for i in range(4):
    print("M", i+1, ": ", decode(ans[i]), sep="")

Números encontrados: 

M1: 6647767204748733612
M2: 6565656565
M3: 4205727813173946050
M4: 2442039398140444127

Mensagens encontradas: 

M1: B/LHJW!=
M2: AAAAA
'.*HN
M4: *]b,
